# LLM API 비교
  - 비용으로 1차적으로 추린 후에 2차적으로 성능 비교

### Gemini API
  - 다른 LLM 라인업 대비 할상 좀 더 저렴한 가격에 제공

In [ ]:
# 환경변수에 GOOGLE_API_KEY 추가 필요
# ~/.zshrc에 export GOOGLE_API_KEY='<API KEY>'
import google.generativeai as genai
import os

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel('gemini-1.5-flash')

In [ ]:
response = model.generate_content("Hi!")
print(response.text)

### Claude API
- OpenAI에 필적하는 성능
  - 2024년 6월 기준 가장 좋은 성능의 모델 = Claude 3.5 Sonnet
- 현재 기준 가장 비싼 모델인 Claude 3 Opus 보유

In [ ]:
# 환경변수에 ANTHROPIC_API_KEY 추가 필요
# ~/.zshrc에 export ANTHROPIC_API_KEY='<API KEY>'
import os

import anthropic

client = anthropic.Anthropic(
    api_key=os.environ['ANTHROPIC_API_KEY']
)

message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    temperature=0.0,
    system="Respond only in Yoda-speak.",
    messages=[
        {"role": "user", "content": "Hi!"}
    ]
)

print(message.content[0].text)

### 동일한 Prompt에 대해 Input / Output 계산
- 비용 계산을 위해서는 Tokens 수를 알아야 함

In [ ]:
from eval import get_eval_data

conversations = get_eval_data()

In [ ]:
print(conversations[0])

#### Gemini Pro & Flash

- 506 / 116 / 622

In [ ]:
prompt = f"""아래 사용자 대화에 대해 3문장 내로 요약해주세요:

{conversations[0]}"""

model = genai.GenerativeModel('gemini-1.5-flash-001')
response = model.generate_content(prompt)
print(response.text)
response.usage_metadata

모델 호출해보기 전에 Prompt가 몇 토큰인 지 미리 알 수 있음

In [ ]:
# 실제로는 여기에 token 1씩 더 추가됨
model.count_tokens(prompt)

동일한 Prompt에 대해 Gemini Pro와 Gemini Flash 둘 다 506 입력 토큰 -> 동일한 Tokenizer를 사용하는 것을 알 수 있음

- 더 많은 단어를 익힌 (보통 더 큰) 모델의 경우 다른 Tokenizer를 써서 입력 토큰 수가 달라질 수 있음

In [ ]:
prompt = f"""아래 사용자 대화에 대해 3문장 내로 요약해주세요:

{conversations[0]}"""

model = genai.GenerativeModel('gemini-1.5-pro-001')
response = model.generate_content(prompt)
print(response.text)
response.usage_metadata

### Anthropic Claude

- 637 / 143 / 780

In [ ]:
import os

import anthropic

client = anthropic.Anthropic(
    api_key=os.environ['ANTHROPIC_API_KEY']
)

message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    temperature=0.0,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(message.content[0].text)
message.usage

In [ ]:
message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=1000,
    temperature=0.0,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(message.content[0].text)
message.usage

In [ ]:
message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1000,
    temperature=0.0,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(message.content[0].text)
message.usage

### OpenAI

- 622 / 177 (GPT 3.5, GPT-4)
- 441 / 76 (GPT-4o)

In [ ]:
from openai import OpenAI


OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)
completion.usage

In [ ]:
from openai import OpenAI


OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model='gpt-4-turbo-2024-04-09',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)
completion.usage

In [ ]:
from openai import OpenAI


OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model='gpt-4o-2024-05-13',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)
completion.usage

#### Input Tokens 수 정리 (Prompt 고정)

- Gemini 506
- Claude 637
- GPT 622
- GPT-4o 441
  - Output은 어느 정도 조정이 가능해서 Input만 비교

- 한글 효율화 측면에서는 GPT-4o > Gemini > GPT-3.5/4 >= Claude 순
  - 다른 Prompt에서는 조금씩 다를 수 있으나, 경향이 크게 달라지지 않을 것으로 예상

#### 비용 계산
- Input Tokens는 바로 위에 수치 사용하고, Output Tokens는 가장 많은 Claude를 200 Tokens로 두고 비례해서 계산
  - ex. Gemini 159 Tokens
- 호출 횟수 별 가격표
  - https://docs.google.com/spreadsheets/d/1L4jfa8F7ifSbp779pInSRkT_JFnT57xsmJvOhvePOk4/edit?usp=sharing

### 모델 별 가격 (1M Token, 2024.06)

Claude 3 Opus / 75.00
Claude 3 Sonnet / 15.00
Claude 3 Haiku / 1.25
Gemini Pro / 10.5 (일정 요청량까지 무료)
Gemini Flash / 1.05 (일정 요청량까지 무료)
GPT-4o / 15
GPT-4 / 30
GPT-3.5 / 1.5

#### 1회 호출 비용

- Claude 3 Opus $0.024 / 33.89원
- Claude 3 Sonnet $0.004 / 6.78원
- Claude 3 Haiku $0.0004 / 0.56원
- Gemini Pro $0.003 / 4.75원
- Gemini Flash $0.0003 / 0.47원
- GPT-4o $0.004 / 5.91원
- GPT-4 $0.012 / 16.67원
- GPT-3.5 $0.0006 / 0.83원

#### 10000회 호출 비용

- Claude 3 Opus $245.55 / 338,859원
- Claude 3 Sonnet $49.11 / 67,772원
- Claude 3 Haiku $4.09 / 5,648원
- Gemini Pro $34.39 / 47,460원
- Gemini Flash $3.44 / 4,746원
- GPT-4o $42.82 / 59,091원
- GPT-4 $120.79 / 166,686원
- GPT-3.5 $6.04 / 8,334원

#### 100만회 호출 비용

- Claude 3 Opus $24,555 / 33,885,900원
- Claude 3 Sonnet $4,911 / 6,777,180원
- Claude 3 Haiku $409.25 / 564,765원
- Gemini Pro $3,439.13 / 4,746,002원
- Gemini Flash $343.91 / 474,600원
- GPT-4o $4,281.92 / 5,909,054원
- GPT-4 $12,078.71 / 16,668,624원
- GPT-3.5 $603.94 / 833,431원

#### 5000만회 호출 비용

- Claude 3 Opus $1,227,750 / 1,694,295,000원
- Claude 3 Sonnet $245,550 / 338,859,000원
- Claude 3 Haiku $20,462.50 / 28,238,250원
- Gemini Pro $171,956.59 / 237,300,099원
- Gemini Flash $17,195.66 / 23,730,010원
- GPT-4o $214,096.15 / 295,452,692원
- GPT-4 $603,935.64 / 833,431,177원
- GPT-3.5 $30,196.78 / 41,671,559원